In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=70d1bbb1-2cb7-45da-8c42-e661010bdf74
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:09<00:00, 136MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from metrics import result

2024-07-01 12:57:58.796033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 12:57:58.796161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 12:57:58.932624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 9093.20it/s]


In [ ]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)

100%|██████████| 10344/10344 [00:00<00:00, 873183.74it/s]
10344it [00:07, 1300.16it/s]
10344it [00:04, 2292.90it/s]


In [ ]:
a1, a2, a3 = np.shape(x_train)
b1, b2, b3 = np.shape(x_test)
x_train = np.reshape(x_train, (a1,a3,a2))
x_test = np.reshape(x_test, (b1,b3,b2))

In [22]:
# Model

def res_block(inp_signal, n_neurons):
    x = tf.keras.layers.Conv1D(n_neurons, padding='same', kernel_size=7)(inp_signal)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv1D(n_neurons, padding='same', kernel_size=7)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = x + tf.keras.layers.Conv1D(n_neurons, kernel_size=1)(inp_signal)
    x = tf.keras.layers.ReLU()(x)
    return x

def get_model():
    input_layer = tf.keras.layers.Input(shape=(100,12))
    x = tf.keras.layers.Conv1D(64, kernel_size=15, strides=2)(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPool1D(2)(x)
    x = res_block(x, 64)
    x = res_block(x, 128)
    x = res_block(x, 256)
    x = res_block(x, 512)
    x = tf.keras.layers.GRU(512)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Dense(7, activation='sigmoid')(x)
    model=tf.keras.models.Model(input_layer, x)
    return model

In [25]:
# Training

## Compiling
def CompModel(model):
    model.compile(optimizer=tf.keras.optimizers.Adam() , 
                loss=tf.keras.losses.BinaryCrossentropy() , metrics=['acc'])
    lr_sch = tf.keras.callbacks.LearningRateScheduler(
        lambda epochs: 1e-3 * 10 ** (-epochs/100.0))
    return model, lr_sch


model = get_model()
model, lr_sch = CompModel(model)

history = model.fit(x_train, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - acc: 0.2424 - loss: 0.6834 - learning_rate: 0.0010
Epoch 2/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - acc: 0.3193 - loss: 0.4066 - learning_rate: 9.7724e-04
Epoch 3/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - acc: 0.3096 - loss: 0.4022 - learning_rate: 9.5499e-04
Epoch 4/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - acc: 0.3673 - loss: 0.3811 - learning_rate: 9.3325e-04
Epoch 5/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - acc: 0.4187 - loss: 0.3590 - learning_rate: 9.1201e-04
Epoch 6/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - acc: 0.5022 - loss: 0.3175 - learning_rate: 8.9125e-04
Epoch 7/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - acc: 0.5478 - loss: 0.2992 - learning_rate: 8.7096e-04
Epoch 8/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - acc: 0.5703 - loss: 0.2842 - learning_rate: 8.5114e-04
Epoch 9/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - acc: 0.5915 - loss: 0.2704 - learning_rate: 8.3176e-04
Epoc

In [26]:
# Evaluating

pred = model.predict(x_test)
prediction = np.zeros(np.shape(pred))
prediction[pred > 0.5] = 1
result(y_test, prediction)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
SNR
accuracy: 83.3 precision: 72.65 sensitivity: 73.49 specificity: 87.68 f1_score: 73.07
_______
AF
accuracy: 86.59 precision: 20.99 sensitivity: 16.35 specificity: 93.74 f1_score: 18.38
_______
IAVB
accuracy: 83.75 precision: 24.74 sensitivity: 17.91 specificity: 92.64 f1_score: 20.78
_______
LBBB
accuracy: 97.96 precision: 82.93 sensitivity: 68.0 specificity: 99.35 f1_score: 74.73
_______
RBBB
accuracy: 91.65 precision: 60.87 sensitivity: 49.12 specificity: 96.44 f1_score: 54.37
_______
SB
accuracy: 85.26 precision: 69.26 sensitivity: 73.21 specificity: 89.24 f1_score: 71.18
_______
STach
accuracy: 90.85 precision: 74.12 sensitivity: 83.63 specificity: 92.67 f1_score: 78.59
_______
accuracy: 54.0 %
AUC: 73.82 %
f1-weighted: 61.89 %
f1-micro: 63.21 %
f1-macro: 55.87 %
